<a href="https://colab.research.google.com/github/kingsgameBen/iShape_tools/blob/main/ishape_styleco_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 爬取日本模特兒網站 stylecorporation
抓取身體資料及所有照片，並從中篩選合格的照片資料


In [ ]:
from datetime import datetime
# 計算年齡
def calculate_age(birth):
    birth_d = datetime.strptime(birth, "%Y-%m")
    today_d = datetime.now()
    if today_d.month> birth_d.month:
        age = today_d.year - birth_d.year
    else:
        age = today_d.year - birth_d.year - 1
    return age

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from urllib.request import urlopen, Request
import shutil
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()

url = "https://stylecorporation.com/profile/"
response = requests.get(url)
response.encoding='utf-8'
# print(response.status_code)
# req = Request(url)
# req.add_header("user-agent","Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36")
# response = urlopen(req)
html = BeautifulSoup(response.text,features="html.parser")
# print(html)

profBox = html.find("div", id="wrapper").find_all("div", class_="profBox")
prof_url_ls = [ url + prof.find("a")["href"] for prof in profBox]
# print(len(prof_url_ls))
dn = "drive/MyDrive/body_shape/styleco/"
if not os.path.exists(dn):
    os.makedirs(dn)
count = 0
for prof_url in prof_url_ls:
    response = requests.get(prof_url)
    response.encoding = 'utf-8'
    html = BeautifulSoup(response.text)
    profThumb_Box = html.find_all("div", class_="profThumb_Box")
    profThumb_ls = [ prof_url[:-10] + thumb.find("a")["href"] for thumb in profThumb_Box]
    detail = html.find("div", class_="satsueikai_RightBox").select("p:nth-of-type(3)")[0].text
    detail_list = detail.split("\r\n")
    brith = "-".join(detail_list[0].split("：")[-1].split(".")[:-1])
    age = calculate_age(brith)
    age = "age：" + str(age)
    details = age + html.find("div", class_="satsueikai_RightBox").text
    # print(detail_list)
    # print(detail)
    print(profThumb_ls)
    print(details)

    count += 1
    prof_dn = dn + str(count) + "/"
    if not os.path.exists(prof_dn):
        os.makedirs(prof_dn)
    for thumb in profThumb_ls:
        fn = prof_dn + thumb.split("/")[-1]
        img = requests.get(thumb, stream = True, verify = False)
        with open(fn, "wb") as f:
            shutil.copyfileobj(img.raw, f)
    prof_dt = prof_dn + "info.txt"
    with open(prof_dt, "w") as txtf:
        txtf.write(details)

